In [34]:
import tensorflow as tf
import glob
import numpy as np
import os
import matplotlib.pyplot as plt

test_or_train = 'train'

face_image_path = '/media/tom/Files/MyPrograms/Python27/tensorflow_CNN/data/new_face_images/' + test_or_train + '/face/'
none_image_path = '/media/tom/Files/MyPrograms/Python27/tensorflow_CNN/data/new_face_images/' + test_or_train + '/none/'

# maybe do some splitting here for validation??
face_image_names = glob.glob(face_image_path + '*.jpeg')
none_image_names = glob.glob(none_image_path + '*.jpeg')
face_image_labels = [1 for x in face_image_names]
none_image_labels = [0 for x in none_image_names]

all_images = face_image_names + none_image_names
all_labels = face_image_labels + none_image_labels

In [35]:
filename_queue = tf.train.string_input_producer(all_images, shuffle=True) #  list of files to read

reader = tf.WholeFileReader()  # initialize a reader to use on filename_queue

In [36]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

raw_image_data = tf.placeholder("float", [60, 80, 3])
raw_image_data_uint8 = tf.cast(raw_image_data, tf.uint8)
image_string_data = tf.image.encode_jpeg(raw_image_data_uint8)

# images and labels array as input
def convert_to(images, labels, name):
    num_examples = labels.shape[0]
    if images.shape[0] != num_examples:
        raise ValueError("Images size %d does not match label size %d." %
                     (images.shape[0], num_examples))
    rows = images.shape[1]
    cols = images.shape[2]

    filename = os.path.join('./new_tfrecord_files/', name + '.tfrecords')
    print('Writing', filename)
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(num_examples):
        #image_raw = images[index].tostring()
        image_data = np.reshape(images[index], [60, 80, 3])
        image_string = sess.run([image_string_data],
                                feed_dict={raw_image_data: image_data})
        example = tf.train.Example(features=tf.train.Features(feature={
            'label': _int64_feature(int(labels[index])),
            'image_raw': _bytes_feature(image_string[0])}))
        writer.write(example.SerializeToString())

In [37]:
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)

In [ ]:
# retrive images and image names from the queue.
# the next image and label can be retrieved by calling .eval()
filename, content = reader.read(filename_queue)

image = tf.image.decode_jpeg(content, channels=3) 
image1 = tf.cast(image, tf.float32)
image2= tf.image.resize_images(image1, [60, 80])

test_or_train = 'prac'

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    
    images = []
    labels = []
    for i in range(5):
        #file = all_images[i]
        #im = image2.eval()
        file, im = sess.run([filename, image2])
        
        #print(file)
        lab = 0
        if b'/face/' in file:
            lab = 1
        print(lab)
        
        im = np.reshape(im, [60, 80, 3])
        
        plt.imshow(255*im)
        plt.show()
        
        images.append(im)
        labels.append([all_labels[i]])
        
    images = np.array(images)
    print(images.shape)
    labels = np.array(labels)
    print(labels.shape)
    #convert_to(images, labels, 'training_images')

    coord.request_stop()
    coord.join(threads)


In [38]:
# finally, convert the images and labels into a .tfrecords file

filename, content = reader.read(filename_queue)

image = tf.image.decode_jpeg(content)
image1 = tf.cast(image, tf.float32)
image2 = tf.image.resize_images(image1, [60, 80])

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    
    num_images = len(all_images)

    print('preparing ' + str(num_images) + ' images.')
    
    images = []
    labels = []
    
    batch_num = 0
    batch_size = 100
    batch_counter = 0
    
    for i in range(num_images):
        batch_counter += 1
        
        file, im = sess.run([filename, image2])
        
        lab = 0
        if b'/face/' in file:
            lab = 1
        
        images.append(im)
        labels.append(lab)
        
        if batch_counter >= batch_size:
            images = np.array(images)
            labels = np.array(labels)
            convert_to(images, labels, test_or_train + str(batch_num))
            batch_counter = 0
            batch_num += 1
            images = []
            labels = []
        
    print('tf records created!!')

    coord.request_stop()
    coord.join(threads)



preparing 19303 images.
Writing ./new_tfrecord_files/train0.tfrecords
Writing ./new_tfrecord_files/train1.tfrecords
Writing ./new_tfrecord_files/train2.tfrecords
Writing ./new_tfrecord_files/train3.tfrecords
Writing ./new_tfrecord_files/train4.tfrecords
Writing ./new_tfrecord_files/train5.tfrecords
Writing ./new_tfrecord_files/train6.tfrecords
Writing ./new_tfrecord_files/train7.tfrecords
Writing ./new_tfrecord_files/train8.tfrecords
Writing ./new_tfrecord_files/train9.tfrecords
Writing ./new_tfrecord_files/train10.tfrecords
Writing ./new_tfrecord_files/train11.tfrecords
Writing ./new_tfrecord_files/train12.tfrecords
Writing ./new_tfrecord_files/train13.tfrecords
Writing ./new_tfrecord_files/train14.tfrecords
Writing ./new_tfrecord_files/train15.tfrecords
Writing ./new_tfrecord_files/train16.tfrecords
Writing ./new_tfrecord_files/train17.tfrecords
Writing ./new_tfrecord_files/train18.tfrecords
Writing ./new_tfrecord_files/train19.tfrecords
Writing ./new_tfrecord_files/train20.tfrecords